# Research Skills: Spatiotemporal Data Analysis
# Worksheet 4 - Fundamentals for Spatial Data Analysis

Sharon Ong, Department of Cognitive Science and Artificial Intelligence – Tilburg University

You will be introduced to the concepts of (geo)spatial data, and more specifically to vector data.


You will then learn how to represent such data in a GeoDataFrame using the GeoPandas library, and the basics to read, explore and visualize such data. And you will exercise all this with some datasets about the Netherlands.

One of the key aspects of geospatial data is how they relate to each other in space. In this worksheet, you will learn the spatial relationships such as spatial weights, spatial lag.   

Spatial Point Process Analysis
1. Visualization of Spatial Point Processes
2. Centrography
3. Density functions: Kernel Density Functions, Quadrant Density Functions
4. Average Nearest Neighbour Distance
5. Ripley's K Functions

Spatial Lattice Data Analysis

6. Creating a GeoDataFrame
7. Handling maps (lattice data) with Python
8. Displaying vector data


### 0. Setup
Please specify in the next cell if you are working from Google Colab or from your own computer. Also indicate if you already have the necessary libraries installed.

In [ ]:
COLAB = True
LIBRARIES_INSTALLED = False

In [ ]:
if COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    # Load the contents of the directory
    !ls
    # Change your working directory to the folder where you stored your files, e.g.
    %cd /content/drive/My Drive/Colab Notebooks/STDA

if not LIBRARIES_INSTALLED:
    !pip install seaborn
    !pip install contextily
    !pip install pointpats
    !pip install mapclassify

from os.path import join

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns
import contextily as ctx
import pointpats

# 1. Visualization of Spatial Point Processes

An example of spatial point process are geo-tagged photos. Geo-tagged photos uploaded to online services is creating new ways for researchers to study and understand cities. Where do people take pictures? When are those pictures taken? Why do certain places attract many more photographers than others? We will use the Tokyo Photographs Dataset for this exercise. This dataset contains geo-tagged Flickr photos from Tokyo. We will treat the phenomena represented in the data as events: photos could be taken of any place in Tokyo, but only certain locations are captured.

The following code loads the dataset. The dataset contains the following information about the sample of 10,000 photographs: the ID of the user who took the photo; the location expressed as latitude and longitude columns; a transformed version of those coordinates expressed in Pseudo Mercator; the timestamp when the photo was taken; and the URL where the picture they refer to is stored online.

In [ ]:
db = pd.read_csv(join('data', 'tokyo_clean.csv'))

# To find out more information about the dataset.
db.info()

Plot the longitude and latitude on a scatter plot with a dot size (`s`) of 0.5.

In [ ]:
#
# Your code goes here
#

The code below creates a scatter plot the seaborn package and displays the map in the background.

In [ ]:
# Create scatter plot with histograms on axes
joint_axes = sns.jointplot(x='longitude', y='latitude', data=db, s=0.75)

# Add basemap
ctx.add_basemap(joint_axes.ax_joint, crs="EPSG:4326", source=ctx.providers.CartoDB.PositronNoLabels)

# Remove axes
joint_axes.ax_joint.set_axis_off()

# 2. Centrography
Centrography is the analysis of centrality (general location and dispersion) in a point pattern. These measures are useful because they allow us to summarize spatial distributions in smaller sets of information (e.g. a single point). The following code computes the mean and median center of the dataset.

In [ ]:
mean_center = pointpats.centrography.mean_center(db[['x', 'y']])
med_center = pointpats.centrography.euclidean_median(db[['x', 'y']])

The following code plots the mean and median points and marginal lines.  

In [ ]:
# Generate scatter plot
joint_axes = sns.jointplot(x='x', y='y', data=db, s=0.75)

# Add mean point and marginal lines
joint_axes.ax_joint.scatter(*mean_center, color='red', marker='x', s=50, label='Mean Center')
joint_axes.ax_marg_x.axvline(mean_center[0], color='red')
joint_axes.ax_marg_y.axhline(mean_center[1], color='red')

# Add median point and marginal lines
joint_axes.ax_joint.scatter(*med_center, color='limegreen', marker='o', s=50, label='Median Center')
joint_axes.ax_marg_x.axvline(med_center[0], color='limegreen')
joint_axes.ax_marg_y.axhline(med_center[1], color='limegreen')

# Legend
joint_axes.ax_joint.legend()

# Add basemap
ctx.add_basemap(joint_axes.ax_joint, source=ctx.providers.CartoDB.Voyager)

# Remove axes
joint_axes.ax_joint.set_axis_off()

A measure of dispersion that is common in centrography is the *standard distance*. This measure provides the average distance away from the center of the point cloud (such as measured by the center of mass). To compute the standard distance, you can use the `std_distance` function in `pointpats.centrography`. Compute the *standard distance* of the dataset.


In [ ]:
#
# Your code goes here
#

Another helpful visualization is the *standard deviational ellipse*, or *standard ellipse* which shows the dispersion and orientation of the dataset. The following code first computes the axes, and rotation of the ellipse. Next, the code shows you how to display the ellipse on the map.



In [ ]:
from matplotlib.patches import Ellipse

major, minor, rotation = pointpats.centrography.ellipse(db[['x','y']])

# Set up figure and axis
fig, ax = plt.subplots(figsize=(9, 9))

# Plot photograph points
ax.scatter(db['x'], db['y'], s=0.75)
ax.scatter(*mean_center, color='red', marker='x', label='Mean Center')
ax.scatter(*med_center, color='limegreen', marker='o', label='Median Center')

# Construct the standard ellipse using matplotlib
ellipse = Ellipse(xy=mean_center, # center the ellipse on our mean center
                  width=major*2, # centrography.ellipse only gives half the axis
                  height=minor*2,
                  angle=np.rad2deg(rotation), # Angles for this are in degrees, not radians
                  facecolor='none', # Aesthetics
                  edgecolor='red',
                  linestyle='--',
                  label='Std. Ellipse')
ax.add_patch(ellipse)
ax.legend()

# Add basemap
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)

# Remove axes
ax.set_axis_off()

In the code block below:

1. Find the users that have posted the most pictures in Tokyo using pandas' `.mode()` method
2. Create a dataframe that only contains the rows from one of these users
3. Find the mean center and euclidean mean of this user
4. Plot the user's photo locations as a scatter plot, the centroids, and the std. ellipse

In [ ]:
#
# Your code goes here
#

# 3 Density Plots
When too many photos are concentrated in some areas of, plotting opaque dots on top of one another can make it hard to discern any pattern and explore its nature.

## 3.1 Kernel Density Plots
Kernel density estimation (KDE): an empirical approximation of the probability density function. Instead of overlaying a grid of squares and count how many points fall within each, we can use a Kernel density estimation (KDE) to lay kernel functions over a grid of points with different weight based on the distance. These counts are then aggregated to generate a global surface with probability. The most common kernel function is the Gaussian one, which applies a normal distribution to weight points.

The code belows generates a number of random points; where there is no clusteirng or dispersion effect. In point pattern analysis, this is known as a *Poisson point process*.

In [ ]:
user = db.loc[db.user_id == '95795770@N00']
coordinates = user[['longitude','latitude']].values
points = pointpats.random.poisson(coordinates, size=len(coordinates))

plt.scatter(points[:, 0], points[:, 1])

The code below creates a kernel density plot with a shading of 50 gradients and a transpancy of 55%


In [ ]:
sns.kdeplot(x=points[:, 0], y=points[:, 1], n_levels=50, fill=True, alpha=0.55, cmap='viridis_r')

Display the Kernel Density Plot for the Tokyo Dataset with the map in the background. Set `n_levels` to 50 and `cmap` to 'viridis_r'.

In [ ]:
#
# Your code goes here
#

Display the Kernel Density Plot for the user with the most uploaded photos, use the same settings as the previous plot, but set `cmap` to 'jet_r'.

In [ ]:
#
# Your code goes here
#

## 3.2 Quadrant Density Plot

Quadrant statistics examines the spatial distribution of points in an area by counting the observations that fall within a given cell. A quadrant statistics examines the *evenness* of the distribution over cells using a $\chi^2$ statistical test common in the analysis of contingency tables.

The code below plots applies quadrant statistics on the previously generated random spatial process and plots the result.


In [ ]:
qstat = pointpats.QStatistic(points)
qstat.plot()

The code belows displays the p value of the $\chi^2$, which shows that p value is large and not significant.  

In [ ]:
qstat.chi2_pvalue

Compute the quadrant statistics on the data from the `coordinates` variable. Display the Quadrant plot and compute the p value of the $\chi^2$.
Is the p value significant?


In [ ]:
#
# Your code goes here
#

# 4. Average Nearest Neighbour
The following code compute the nearest neighbour for the random point process and draws an arrow from each point to its nearest neighbour.


In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(8, 4), sharex=True, sharey=True, layout='constrained')

axs[0].scatter(*points.T, color='red', marker='.')
axs[1].scatter(*points.T, color='red', zorder=100, marker='.', label='Point')

pp = pointpats.PointPattern(points)
nn_ixs, nn_ds = pp.knn(1)

first = True
for coord, nn_ix, nn_d in zip(points, nn_ixs, nn_ds):
    dx, dy = points[nn_ix].squeeze() - coord
    if first:
        first = False
        arrow = axs[1].arrow(*coord, dx, dy, length_includes_head=True, facecolor='k', width=0.001, label='Nearest Neighbor of Point')
    else:
        arrow = axs[1].arrow(*coord, dx, dy, length_includes_head=True, facecolor='k', width=0.003, head_width=0.005)

axs[0].set_axis_off()
axs[1].set_axis_off()
axs[1].legend(bbox_to_anchor=(0.7, -0.1))
plt.show()

# 5. K(d) functions

K(d) fucntions summarizes the density between points for all distances. It consists of dividing the mean of  sum of the number of points at different distance lags for each point by the entire area. The following code applies a K function on the random point process and plots it.

In [ ]:
dist, k = pointpats.k(points)

plt.plot(dist, k)
plt.xlabel('Distance (km)')
plt.ylabel('K')

Compute the K function for the single user coordinates from the Tokyo dataset   



In [ ]:
#
# Your code goes here
#

# 6. Creating a GeoDataFrame
The following loads a csv file of some major cities in North Brabant and converts the dataset to a GeoDataFrame.  
1. Inspect the first 5 rows of the cities dataframe and the  with the head() method. Do you see the columns with coordinates?
2. Visualize the locations of the cities (you may use the `gdf.plot()` or matplotlib)

In [ ]:
cities = pd.read_csv(join('data', 'nl_noord-brabant_main.csv'))
print(cities.head())
gdf = gpd.GeoDataFrame(cities, geometry=gpd.points_from_xy(cities.lng, cities.lat))

#
# Your code goes here
#

# 7. Handling Choropleth maps with Python
GeoDataFrames can be used to store simple geographical features, along with their non-spatial attribute. The GeoPandas library has functions to read, explore and visualize such data.  These geographical features include points (therefore addresses and locations), line strings (therefore streets, highways and boundaries), polygons (countries, provinces, tracts of land), and multi-part collections of these type. The following code loads a shapefile of all the municipalities in the Netherlands.

In [ ]:
gemeenten = gpd.read_file(join('data', 'wijkbuurtkaart_2023_v1', 'gemeenten_2023_v1.shp'))

# Display the shapes (e.g. polygons and multi-polygons) with a transparency of 0.5,
# green face color and black edges
ax = gemeenten.plot(figsize=(12, 12), alpha=0.5, facecolor='g', edgecolor='w')

display(gemeenten.head())

# Annotate the polygon with its index
gemeenten['coords'] = gemeenten['geometry'].apply(lambda x: x.representative_point().coords[:])

display(gemeenten.head())

gemeenten['coords'] = [coords[0] for coords in gemeenten['coords']]
for idx, row in gemeenten.iterrows():
    plt.annotate(idx, xy=row['coords'], horizontalalignment='center')


If you call a single row of the geometry column, it'll return a small plot with the shape. Try other row numbers.  

In [ ]:
gemeenten.loc[17, 'geometry']

#
# Your code goes here
#

# 7.2 Data mapping

A choropleth for categorical variables simply assigns a different color to every potential value in the series. The main requirement in this case is then for the color scheme to reflect the fact that different values are not ordered or follow a particular scale.

We can create categorical choropleths with geopandas. The following code displays each polygon as whether it is a polygon representing land, water with a legend.


In [ ]:
display(gemeenten.head())

gemeenten.plot(column='H2O', cmap='jet', categorical=True, legend=True, legend_kwds={'loc': 'upper left'})

Remove all the polygons representing water and find the number of municipalities in 2023.

In [ ]:
#
# Your code goes here
#

We can create a map that displays the polygons with colors assigned by some feature values. The following code calculates the area of each polygon. These values can be assigned a color specified by the selected colormap.  Then a new geopandas column called 'area' is created and assigned the area information.  You will see that smaller polygons have lower size and vice versa.  

In [ ]:
gemeenten.loc[:, 'area'] = gemeenten.geometry.area / 100000
gemeenten.plot(figsize=(9, 9), column='area', cmap='jet', legend=True)

# 7.3 Quantiles
One solution to obtain a more balanced classification scheme is using quantiles. This, by definition, assigns the same amount of values to each bin: the entire series is laid out in order and break points are assigned in a way that leaves exactly the same amount of observations between each of them. This "observation-based" approach contrasts with the "value-based" method of equal intervals and, although it can obscure the magnitude of extreme values, it can be more informative in cases with skewed distributions.

In [ ]:
gemeenten.plot(figsize=(9, 9), column='area', scheme='QUANTILES', k=3, cmap='jet', legend=True, legend_kwds={'loc': 'upper left'})